In [ ]:
!pip install flask-ngrok

In [2]:
import os
os.chdir('/content/drive/MyDrive/styletrasfer')

In [ ]:
from flask import Flask , render_template , request , url_for
from flask_ngrok import run_with_ngrok
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import os

app = Flask(__name__)
run_with_ngrok(app)


upload_path = "static/upload"
print("**********************Start loading the model**********************")
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("**********************Model loading is done**********************")

def load_img(img_path):
    image = tf.keras.preprocessing.image.load_img(img_path)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0
    return image[tf.newaxis , ]

def style_transfer(img1_path , img2_path):
    my_img = load_img(img1_path)
    style_img = load_img(img2_path)
    new_img = model(tf.constant(my_img) , tf.constant(style_img))
    new_img_get = np.array(new_img[0][0] * 255 , dtype=np.uint8)
    
    return new_img_get

@app.route("/" , methods = ['POST' , "GET"])
def home():
    # Get images from html form
    if request.method == "POST":
        if 'file1' not in request.files or 'file2' not in request.files:
            print('No file part')
            return render_template("home.html")
        file1 = request.files['file1']
        file2 = request.files['file2']
        filename1 = file1.filename
        filename2 = file2.filename
        filename1 = os.path.join(upload_path , filename1)
        filename2 = os.path.join(upload_path , filename2)
        file1.save(filename1)
        file2.save(filename2)

        style_img = style_transfer(filename1,filename2)
        img_name = "upload_img"+str(time.time()).replace(".","_")+".jpg"
        new_img_path = os.path.join(upload_path , img_name)
        try:
            os.remove(new_img_path)
        except:
            pass
        cv2.imwrite(new_img_path , style_img)
        return render_template("home.html" , show_result = True , image = [filename1,filename2,new_img_path])
    return render_template("home.html" , show_result = False)

app.run()

**********************Start loading the model**********************
**********************Model loading is done**********************
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1e1b0972420e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-23-bf7d0875c250>", line 57, in home
    return render_template("home.html" , show_result = False)
  File "/usr/local

In [13]:
import time

In [18]:
str(time.time()).replace(".","_")

'1619431573_2767663'